In [ ]:
# Import
import os
import sys
import shutil

import math 

try:
    import openmm as mm
    import openmm.app as app
    import openmm.unit as unit
except ImportError:
    import simtk.openmm as mm
    import simtk.openmm.app as app
    import simtk.unit as unit

root_filepath = "C:/Users/Thomas/Documents/workspace/research/OpenABC_RNA" # you will need to set the filepath
if root_filepath not in sys.path:
    sys.path.insert(0, root_filepath)

tests_filepath = os.path.join(root_filepath, "tests")
pdbs_filepath = os.path.join(tests_filepath, "pdbs")
runs_filepath = os.path.join(tests_filepath, "runs")

from openabc.forcefields.parsers import SMOGParser
from openabc.forcefields import SMOGSSPNModel
from openabc.utils.insert import insert_molecules
import matplotlib.pyplot as plt

def get_run_id(runs_filepath):
    files = [f for f in os.listdir(runs_filepath) if os.path.isdir(os.path.join(runs_filepath, f))]
    numbers = []
    for file in files:
        try:
            number = int(file)
            numbers.append(number)
        except ValueError:
            continue
    largest_number = max(numbers, default=None)
    if largest_number is not None:
        return f"{largest_number + 1:03d}"
    else:
        return "001"

In this tutorial, we will look at how to run simulations with multiple copies of the same system, e.g. in a biomolecular condensate. The setup remains largely the same. Note the new parameter `number_of_rnas`, which indicates how many identical copies there will be of the molecule you choose, as well as the dimensions of the periodic boundary box `box_a`, `box_b`, `box_c`. If the box size is too small relative to the number of RNAs, they may take a very long time to insert, or may fail to insert.

In [ ]:
# Select which system you wish to simulate.
system = "P5abc"
raw_rna_pdb_filepath = os.path.join(pdbs_filepath, system + '.pdb')
cut_rna_pdb_filepath = os.path.join(pdbs_filepath, system + '_cut.pdb')
cg_rna_pdb_filepath = os.path.join(pdbs_filepath, system + '_cg.pdb')
cg_multiple_rnas_pdb_filepath = os.path.join(pdbs_filepath, system + '_multiple_cg.pdb')

In [ ]:
# Run Settings
platform_name = 'CUDA'

number_of_rnas = 2
simulation_time = 0.02 * unit.nanosecond
report_interval = 500
timestep = 10 * unit.femtosecond
temperature = 300*unit.kelvin
friction_coeff = 1.0 / unit.picosecond

box_a, box_b, box_c = 50, 50, 50

This time, we will create two models. One will represent the single RNA, and one will represent the assembly of RNAs. The `insert_molecules` function requires a structure file for the system you wish to insert, so we must create a model of the single RNA to write its coordinates to a PDB file. Then the setup continues similarly to that of `single_rna.ipynb`.

In [ ]:
# Create Model
rna_data = SMOGParser.from_atomistic_pdb(raw_rna_pdb_filepath, cg_rna_pdb_filepath, cut_rna_pdb_filepath, mol_type='rna', cg_type='phosphorus', default_parse=False)
rna_data.parse_mol(get_native_pairs=True, mol_type='rna', bonded_energy_scale=3.75)
single_rna_model = SMOGSSPNModel()
single_rna_model.append_mol(rna_data)
single_rna_model.atoms_to_pdb(cg_rna_pdb_filepath)
multiple_rnas_model = SMOGSSPNModel()

for i in range(number_of_rnas):
    multiple_rnas_model.append_mol(rna_data)
insert_molecules(cg_rna_pdb_filepath, cg_multiple_rnas_pdb_filepath, n_mol=number_of_rnas, box=[box_a, box_b, box_c])
multiple_rnas_model.atoms_to_pdb(cg_rna_pdb_filepath)

top = app.PDBFile(cg_multiple_rnas_pdb_filepath).getTopology()
multiple_rnas_model.create_system(top, box_a=box_a, box_b=box_b, box_c=box_c)
multiple_rnas_model.add_sspn_bonds(force_group=1)
multiple_rnas_model.add_sspn_angles(threshold=160*math.pi/180, clip=True, force_group=2)
multiple_rnas_model.add_sspn_dihedrals(force_group=3)
multiple_rnas_model.add_native_pairs(force_group=4)
multiple_rnas_model.exclusions = multiple_rnas_model.sspn_exclusions
multiple_rnas_model.add_elec_switch(force_group=6)
multiple_rnas_model.add_smog_vdwl(force_group=11)


In [ ]:
# Initialize
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
multiple_rnas_model.init_coord = app.PDBFile(cg_rna_pdb_filepath).getPositions()
run_id = get_run_id(runs_filepath)
run_filepath = os.path.join(runs_filepath, run_id)
os.makedirs(run_filepath, exist_ok=True)
shutil.copy(cg_rna_pdb_filepath, os.path.join(run_filepath,'structure.pdb'))
trajectory_filepath = os.path.join(run_filepath, "trajectory.dcd")

initial_coordinates = app.PDBFile(cg_multiple_rnas_pdb_filepath).getPositions()
multiple_rnas_model.set_simulation(integrator, platform_name=platform_name, init_coord=initial_coordinates)
multiple_rnas_model.add_reporters(report_interval, output_dcd=trajectory_filepath, report_dcd=True, report_state_data=True)
multiple_rnas_model.simulation.minimizeEnergy()

In [ ]:
# Run Simulation
raw_steps = simulation_time / timestep
report_number = math.ceil(raw_steps / report_interval)
steps = int(report_number * report_interval)
multiple_rnas_model.simulation.step(steps)

for reporter in multiple_rnas_model.simulation.reporters:
    reporter._out.flush()
multiple_rnas_model.simulation.reporters = None

For questions contact trthorn3@ncsu.edu